# S10 — Embeddings & Moteurs de Recherche Vectorielle (RAG Intro)

## 🎯 Objectifs
- Comprendre les embeddings et leur création
- Maîtriser la recherche de similarité (nearest neighbour)
- Implémenter un index FAISS local
- Évaluer recall et precision du retrieval

## 📋 Contenu
1. Création d'embeddings
2. Similarity search et métriques de distance
3. Index types (Flat, IVFPQ)
4. Implémentation avec FAISS
5. Évaluation des performances

## 1. Installation et Configuration

In [ ]:
# Installation des dépendances
# !pip install faiss-cpu openai sentence-transformers pandas numpy scikit-learn

In [ ]:
import os
import numpy as np
import pandas as pd
import faiss
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
from typing import List, Tuple
import pickle

print("✅ Bibliothèques importées")

## 2. Création du Dataset

Nous allons créer un dataset simulé de documents techniques pour la démonstration.

In [ ]:
# Dataset de documents (simulé)
documents = [
    {"id": 1, "text": "Le machine learning est une branche de l'intelligence artificielle.", "category": "ML"},
    {"id": 2, "text": "Les réseaux de neurones profonds sont utilisés pour l'apprentissage profond.", "category": "DL"},
    {"id": 3, "text": "Python est un langage de programmation populaire pour la data science.", "category": "Programming"},
    {"id": 4, "text": "FastAPI permet de créer des APIs REST rapidement avec Python.", "category": "Programming"},
    {"id": 5, "text": "Les transformers révolutionnent le traitement du langage naturel.", "category": "NLP"},
    {"id": 6, "text": "BERT est un modèle pré-entraîné basé sur l'architecture transformer.", "category": "NLP"},
    {"id": 7, "text": "Le gradient descent est un algorithme d'optimisation fondamental.", "category": "ML"},
    {"id": 8, "text": "Les embeddings capturent la sémantique des mots dans un espace vectoriel.", "category": "NLP"},
    {"id": 9, "text": "Docker permet de conteneuriser des applications pour un déploiement facile.", "category": "DevOps"},
    {"id": 10, "text": "Kubernetes orchestre des conteneurs à grande échelle.", "category": "DevOps"},
    {"id": 11, "text": "Les CNNs sont particulièrement efficaces pour la vision par ordinateur.", "category": "DL"},
    {"id": 12, "text": "Le fine-tuning adapte un modèle pré-entraîné à une tâche spécifique.", "category": "ML"},
    {"id": 13, "text": "GPT-4 est un modèle de langage génératif développé par OpenAI.", "category": "NLP"},
    {"id": 14, "text": "Les API REST utilisent HTTP pour communiquer entre services.", "category": "Programming"},
    {"id": 15, "text": "L'attention est le mécanisme clé des architectures transformer.", "category": "DL"},
    {"id": 16, "text": "Pandas est une bibliothèque Python pour l'analyse de données.", "category": "Programming"},
    {"id": 17, "text": "Le RAG combine retrieval et génération pour améliorer les LLMs.", "category": "NLP"},
    {"id": 18, "text": "FAISS est une bibliothèque pour la recherche de similarité vectorielle.", "category": "ML"},
    {"id": 19, "text": "Les bases de données vectorielles stockent et recherchent des embeddings.", "category": "ML"},
    {"id": 20, "text": "Le tokenisation découpe le texte en unités traitables par les modèles.", "category": "NLP"},
]

df = pd.DataFrame(documents)
print(f"📊 Dataset créé: {len(df)} documents")
print(df.head())

## 3. Génération d'Embeddings

### 3.1 Chargement du modèle d'embeddings

In [ ]:
# Charger un modèle d'embeddings multilingue
# Options: 'paraphrase-multilingual-MiniLM-L12-v2', 'distiluse-base-multilingual-cased-v2'
embedding_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

print(f"✅ Modèle chargé: {embedding_model.get_sentence_embedding_dimension()} dimensions")

### 3.2 Création des embeddings

In [ ]:
def create_embeddings(texts: List[str], model: SentenceTransformer) -> np.ndarray:
    """
    Créer des embeddings pour une liste de textes
    """
    embeddings = model.encode(texts, show_progress_bar=True)
    return np.array(embeddings).astype('float32')

# Générer les embeddings
texts = df['text'].tolist()
embeddings = create_embeddings(texts, embedding_model)

print(f"✅ Embeddings créés: shape = {embeddings.shape}")
print(f"   - {embeddings.shape[0]} documents")
print(f"   - {embeddings.shape[1]} dimensions")

## 4. Métriques de Similarité

### 4.1 Cosine Similarity

In [ ]:
def compute_cosine_similarity(embedding1: np.ndarray, embedding2: np.ndarray) -> float:
    """
    Calculer la similarité cosinus entre deux embeddings
    """
    return cosine_similarity([embedding1], [embedding2])[0][0]

# Exemple: Comparer deux documents
doc1_emb = embeddings[0]  # "Le machine learning..."
doc2_emb = embeddings[1]  # "Les réseaux de neurones..."
doc3_emb = embeddings[2]  # "Python est un langage..."

sim_1_2 = compute_cosine_similarity(doc1_emb, doc2_emb)
sim_1_3 = compute_cosine_similarity(doc1_emb, doc3_emb)

print(f"Similarité entre doc1 et doc2 (tous deux ML): {sim_1_2:.4f}")
print(f"Similarité entre doc1 et doc3 (ML vs Programming): {sim_1_3:.4f}")

### 4.2 Matrice de similarité

In [ ]:
# Calculer la matrice de similarité pour les 10 premiers documents
sample_embeddings = embeddings[:10]
similarity_matrix = cosine_similarity(sample_embeddings)

# Visualiser
plt.figure(figsize=(10, 8))
plt.imshow(similarity_matrix, cmap='YlOrRd', aspect='auto')
plt.colorbar(label='Cosine Similarity')
plt.xlabel('Document ID')
plt.ylabel('Document ID')
plt.title('Matrice de Similarité (10 premiers documents)')
plt.xticks(range(10), [f"Doc {i+1}" for i in range(10)], rotation=45)
plt.yticks(range(10), [f"Doc {i+1}" for i in range(10)])
plt.tight_layout()
plt.show()

## 5. FAISS: Index de Recherche Vectorielle

### 5.1 Index Flat (Brute Force)

In [ ]:
def create_faiss_index_flat(embeddings: np.ndarray) -> faiss.IndexFlatL2:
    """
    Créer un index FAISS Flat (recherche exhaustive)
    """
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)  # L2 distance
    index.add(embeddings)  # Ajouter les vecteurs
    return index

# Créer l'index
index_flat = create_faiss_index_flat(embeddings)

print(f"✅ Index Flat créé")
print(f"   - Nombre de vecteurs: {index_flat.ntotal}")
print(f"   - Dimension: {index_flat.d}")

### 5.2 Recherche de similarité

In [ ]:
def search_similar(query: str, index: faiss.Index, model: SentenceTransformer, 
                   df: pd.DataFrame, k: int = 5) -> pd.DataFrame:
    """
    Rechercher les k documents les plus similaires à une query
    """
    # Créer l'embedding de la query
    query_embedding = model.encode([query]).astype('float32')
    
    # Rechercher les k plus proches voisins
    distances, indices = index.search(query_embedding, k)
    
    # Créer un DataFrame des résultats
    results = []
    for i, (dist, idx) in enumerate(zip(distances[0], indices[0])):
        results.append({
            "rank": i + 1,
            "id": df.iloc[idx]['id'],
            "text": df.iloc[idx]['text'],
            "category": df.iloc[idx]['category'],
            "distance": dist,
            "similarity": 1 / (1 + dist)  # Approximation de similarité
        })
    
    return pd.DataFrame(results)

# Test de recherche
query = "Comment fonctionne le deep learning?"
print(f"🔍 Query: {query}\n")

results = search_similar(query, index_flat, embedding_model, df, k=5)
print("📊 Top 5 résultats:")
print(results[['rank', 'text', 'category', 'similarity']].to_string(index=False))

### 5.3 Tests avec différentes queries

In [ ]:
test_queries = [
    "Qu'est-ce qu'un transformer?",
    "Comment déployer une application?",
    "Python pour l'analyse de données",
    "Recherche vectorielle et embeddings"
]

for query in test_queries:
    print(f"\n{'='*80}")
    print(f"🔍 Query: {query}")
    print('='*80)
    
    results = search_similar(query, index_flat, embedding_model, df, k=3)
    
    for _, row in results.iterrows():
        print(f"{row['rank']}. [{row['category']}] {row['text'][:60]}... (sim: {row['similarity']:.3f})")

## 6. Index IVFPQ (Optimisé pour Large Scale)

### 6.1 Création d'un index IVF (Inverted File)

In [ ]:
def create_faiss_index_ivf(embeddings: np.ndarray, nlist: int = 10) -> faiss.IndexIVFFlat:
    """
    Créer un index FAISS IVF (plus rapide pour large scale)
    
    Args:
        embeddings: Vecteurs à indexer
        nlist: Nombre de clusters (voronoi cells)
    """
    dimension = embeddings.shape[1]
    
    # Créer le quantizer (index flat pour les centroids)
    quantizer = faiss.IndexFlatL2(dimension)
    
    # Créer l'index IVF
    index = faiss.IndexIVFFlat(quantizer, dimension, nlist)
    
    # Entraîner l'index (clustering)
    index.train(embeddings)
    
    # Ajouter les vecteurs
    index.add(embeddings)
    
    return index

# Pour notre petit dataset, utilisons nlist=5
index_ivf = create_faiss_index_ivf(embeddings, nlist=5)

print(f"✅ Index IVF créé")
print(f"   - Nombre de vecteurs: {index_ivf.ntotal}")
print(f"   - Nombre de clusters: {index_ivf.nlist}")

### 6.2 Comparaison Flat vs IVF

In [ ]:
import time

def benchmark_search(query: str, index: faiss.Index, model: SentenceTransformer, k: int = 5):
    """
    Mesurer le temps de recherche
    """
    query_embedding = model.encode([query]).astype('float32')
    
    start = time.time()
    distances, indices = index.search(query_embedding, k)
    elapsed = time.time() - start
    
    return elapsed * 1000  # en ms

# Benchmark
query = "Qu'est-ce que le machine learning?"

time_flat = benchmark_search(query, index_flat, embedding_model)
time_ivf = benchmark_search(query, index_ivf, embedding_model)

print(f"⏱️ Temps de recherche:")
print(f"   - Flat (exact): {time_flat:.3f} ms")
print(f"   - IVF (approx): {time_ivf:.3f} ms")
print(f"   - Speedup: {time_flat/time_ivf:.2f}x")
print(f"\n⚠️ Note: Sur de petits datasets, Flat peut être plus rapide.")
print(f"   IVF devient intéressant à partir de 10k-100k+ vecteurs.")

## 7. Évaluation: Recall et Precision

### 7.1 Création d'un ground truth

In [ ]:
# Définir des queries avec leurs documents pertinents attendus (ground truth)
ground_truth = [
    {
        "query": "Qu'est-ce qu'un transformer?",
        "relevant_ids": [5, 6, 15]  # Docs sur transformers, BERT, attention
    },
    {
        "query": "Comment utiliser Python?",
        "relevant_ids": [3, 4, 16]  # Docs sur Python, FastAPI, Pandas
    },
    {
        "query": "Machine learning et optimisation",
        "relevant_ids": [1, 7, 12]  # Docs sur ML, gradient descent, fine-tuning
    },
    {
        "query": "Recherche vectorielle",
        "relevant_ids": [8, 18, 19]  # Docs sur embeddings, FAISS, vector DB
    },
]

print("✅ Ground truth défini pour l'évaluation")

### 7.2 Calcul de Recall@k et Precision@k

In [ ]:
def evaluate_retrieval(query: str, relevant_ids: List[int], index: faiss.Index, 
                      model: SentenceTransformer, df: pd.DataFrame, k: int = 5):
    """
    Calculer Recall@k et Precision@k
    """
    # Récupérer les résultats
    query_embedding = model.encode([query]).astype('float32')
    _, indices = index.search(query_embedding, k)
    
    # IDs récupérés
    retrieved_ids = [df.iloc[idx]['id'] for idx in indices[0]]
    
    # Calculer les métriques
    relevant_retrieved = set(relevant_ids) & set(retrieved_ids)
    
    recall = len(relevant_retrieved) / len(relevant_ids) if relevant_ids else 0
    precision = len(relevant_retrieved) / k
    
    return {
        "query": query,
        "k": k,
        "relevant_ids": relevant_ids,
        "retrieved_ids": retrieved_ids,
        "relevant_retrieved": list(relevant_retrieved),
        "recall": recall,
        "precision": precision
    }

# Évaluer toutes les queries
evaluation_results = []

for item in ground_truth:
    result = evaluate_retrieval(
        query=item["query"],
        relevant_ids=item["relevant_ids"],
        index=index_flat,
        model=embedding_model,
        df=df,
        k=5
    )
    evaluation_results.append(result)
    
    print(f"\n🔍 Query: {result['query']}")
    print(f"   Relevant: {result['relevant_ids']}")
    print(f"   Retrieved: {result['retrieved_ids']}")
    print(f"   ✅ Found: {result['relevant_retrieved']}")
    print(f"   📊 Recall@{result['k']}: {result['recall']:.2%}")
    print(f"   📊 Precision@{result['k']}: {result['precision']:.2%}")

### 7.3 Métriques moyennes

In [ ]:
avg_recall = np.mean([r['recall'] for r in evaluation_results])
avg_precision = np.mean([r['precision'] for r in evaluation_results])

print(f"\n{'='*80}")
print(f"📊 MÉTRIQUES MOYENNES")
print(f"{'='*80}")
print(f"Recall@5:    {avg_recall:.2%}")
print(f"Precision@5: {avg_precision:.2%}")

# Visualisation
fig, ax = plt.subplots(1, 2, figsize=(12, 4))

queries = [r['query'][:30] + '...' for r in evaluation_results]
recalls = [r['recall'] for r in evaluation_results]
precisions = [r['precision'] for r in evaluation_results]

ax[0].barh(queries, recalls, color='skyblue')
ax[0].set_xlabel('Recall@5')
ax[0].set_title('Recall par Query')
ax[0].set_xlim(0, 1)

ax[1].barh(queries, precisions, color='lightcoral')
ax[1].set_xlabel('Precision@5')
ax[1].set_title('Precision par Query')
ax[1].set_xlim(0, 1)

plt.tight_layout()
plt.show()

## 8. Sauvegarde et Chargement de l'Index

### 8.1 Sauvegarde

In [ ]:
def save_index(index: faiss.Index, embeddings: np.ndarray, df: pd.DataFrame, 
               index_path: str = "faiss_index.bin", 
               data_path: str = "documents.pkl"):
    """
    Sauvegarder l'index FAISS et les données associées
    """
    # Sauvegarder l'index FAISS
    faiss.write_index(index, index_path)
    
    # Sauvegarder les données (documents + embeddings)
    data = {
        "documents": df,
        "embeddings": embeddings
    }
    with open(data_path, 'wb') as f:
        pickle.dump(data, f)
    
    print(f"✅ Index sauvegardé: {index_path}")
    print(f"✅ Données sauvegardées: {data_path}")

# Sauvegarder
save_index(index_flat, embeddings, df)

### 8.2 Chargement

In [ ]:
def load_index(index_path: str = "faiss_index.bin", 
               data_path: str = "documents.pkl"):
    """
    Charger l'index FAISS et les données associées
    """
    # Charger l'index
    index = faiss.read_index(index_path)
    
    # Charger les données
    with open(data_path, 'rb') as f:
        data = pickle.load(f)
    
    print(f"✅ Index chargé: {index.ntotal} vecteurs")
    print(f"✅ Données chargées: {len(data['documents'])} documents")
    
    return index, data['documents'], data['embeddings']

# Test de chargement
loaded_index, loaded_df, loaded_embeddings = load_index()

# Vérifier que ça fonctionne
test_query = "Qu'est-ce que l'IA?"
test_results = search_similar(test_query, loaded_index, embedding_model, loaded_df, k=3)
print(f"\n🔍 Test query: {test_query}")
print(test_results[['rank', 'text']].to_string(index=False))

## 9. Exercices Pratiques

### Exercice 1: Ajouter de nouveaux documents
1. Créer 10 nouveaux documents
2. Générer leurs embeddings
3. Les ajouter à l'index existant
4. Tester la recherche

### Exercice 2: Optimiser les hyperparamètres
1. Tester différentes valeurs de `nlist` pour IVF
2. Mesurer l'impact sur la vitesse et le recall
3. Trouver le meilleur compromis

### Exercice 3: Implémenter un filtre de métadonnées
1. Modifier `search_similar` pour filtrer par catégorie
2. Exemple: "Chercher uniquement dans les docs NLP"
3. Comparer les résultats avec/sans filtre

### Exercice 4: Évaluation avancée
1. Implémenter Mean Reciprocal Rank (MRR)
2. Calculer NDCG@k
3. Comparer Flat vs IVF sur ces métriques

## 10. Concepts Avancés

### 10.1 Types d'index FAISS

| Type | Description | Usage |
|------|-------------|-------|
| **Flat** | Brute force, exact | < 10K vecteurs, besoin de précision maximale |
| **IVF** | Inverted file, approximatif | 10K - 10M vecteurs |
| **IVFPQ** | IVF + Product Quantization | 10M+ vecteurs, compression |
| **HNSW** | Hierarchical NSW graph | Très rapide, mémoire élevée |

### 10.2 Métriques de distance
- **L2 (Euclidean)**: Distance euclidienne classique
- **Inner Product**: Produit scalaire (pour vecteurs normalisés = cosine)
- **Cosine**: Angle entre vecteurs

### 10.3 Librairies alternatives
- **Milvus**: Vector DB distribuée, production-ready
- **Weaviate**: Vector DB avec GraphQL
- **Pinecone**: Vector DB managée (cloud)
- **Qdrant**: Vector DB en Rust, performante

## 📚 Ressources

- [FAISS Documentation](https://github.com/facebookresearch/faiss/wiki)
- [Sentence Transformers](https://www.sbert.net/)
- [Understanding Vector Search](https://www.pinecone.io/learn/vector-search/)
- [Embeddings Guide](https://platform.openai.com/docs/guides/embeddings)

## ✅ Checklist

- [ ] Embeddings créés pour tous les documents
- [ ] Index FAISS Flat implémenté
- [ ] Index FAISS IVF implémenté
- [ ] Recherche de similarité testée
- [ ] Recall et Precision calculés
- [ ] Index sauvegardé localement
- [ ] Benchmark de performance effectué

---

**Session S10 complétée! 🚀**